In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib auto

cv2.__version__

Using matplotlib backend: MacOSX


'3.1.0'

In [2]:
colList = []
def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    global colList
    colList.append(event.x)

def getPoint(x,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x,y]

def getLineSegment(x1,x2,y,img):
    temp = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return temp[x1:x2,y]

In [3]:
frameOffset = 300
bufLen = 5

# grab and display a frame during run
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,frameOffset)
ret,frame = cap.read()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(frame)

# mouse click on columns to histogram
cid = fig.canvas.mpl_connect('button_press_event', onclick);


button=1, x=262, y=103, xdata=234.338710, ydata=416.274194
button=1, x=268, y=103, xdata=242.080645, ydata=416.274194
button=1, x=272, y=103, xdata=247.241935, ydata=416.274194
button=1, x=279, y=104, xdata=256.274194, ydata=414.983871
button=1, x=287, y=105, xdata=266.596774, ydata=413.693548
button=1, x=298, y=104, xdata=280.790323, ydata=414.983871
button=1, x=306, y=104, xdata=291.112903, ydata=414.983871


In [5]:
# get these with mouse clicks later
x1 = 300
x2 = 470
numSubBuff = len(colList)
cap = cv2.VideoCapture('test.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES,0)

ret,frame = cap.read()
pixel = getLineSegment(x1,x2,colList[0],frame) 
buffer = np.tile(pixel, [2,numSubBuff,1])
subBuff = buffer[1,:,:].copy()

ret,frame = cap.read()
while ret:

    # create sub buffer
    for ii in range(0,numSubBuff):
        y = colList[ii]
        new = getLineSegment(x1,x2,y,frame) 
        subBuff[ii,:] = new    
    
    # append to main buffer
    buffer = np.append(buffer, np.expand_dims(subBuff, axis=0), axis=0)
    
    # read next frame
    ret,frame = cap.read()

cap.release()



True

KeyboardInterrupt: 

In [10]:
# mask traces (remove noise) and aggregate
noiseThreshold = 90
buf = buffer>noiseThreshold
out = np.sum(np.sum(buf, axis=1), axis=1)


In [20]:
plt.plot(ddy)

In [9]:
# manually set noise threshold
plt.imshow(buffer[0:1000,6,:])
plt.colorbar()

In [13]:
# mask traces (remove noise) and aggregate
noiseThreshold = 90
buf = buffer>noiseThreshold
out = np.sum(np.sum(buf, axis=1), axis=1)

# low pass filter to smooth
N=10
Fc=40
Fs=1600
import scipy.signal as sp

# use firwin with arbitrary Fs
h=sp.firwin( numtaps=N, cutoff=40, nyq=Fs/2)
y=sp.lfilter( h, 1.0, out) # 'x' is the time-series data you are filtering


# count number of positive inflextions points (first derivative zero / second neg)
dy = np.diff(y)
dy[dy>0] = 1
dy[dy<0] = -1
ddy = np.diff(dy)
ddy[ddy>0] = 0
np.count_nonzero(ddy)

209